# SI 330: Sample Midterm


You have 80 minutes to complete this exam.

# Question 1: Data Cleaning
Listed below are all the 36 known earthquakes with an estimated magnitude of 8.5 or higher since the year 1500. But like most stuff from Wikipedia, it's in need of some data cleaning in order to be useful.

## Part A
We need to transform this wikipedia dataset into a well-formated table with the following columns:
1. **year**: The year of each earthquake (1960, 1964, 2004, etc.)
2. **month**: The month of each earthquake (May, March, December, etc.)
3. **day**: The day of each earthquake (22, 27, 26, etc.)
4. **magnitude**: The magnitude of each earthquake. If the magnitude has a range format (9.4-9.6), compute the average magnitude (9.5). Citation numbers ([2], [3], etc.) should also be removed.

In [1]:
import pandas as pd
import numpy as np
data = pd.read_html("https://en.wikipedia.org/wiki/Lists_of_earthquakes")[2]
data.head(5)

,Rank,Date,Location,Event,Magnitude
0,1,"May 22, 1960","Valdivia, Chile",1960 Valdivia earthquake,9.4–9.6
1,2,"March 27, 1964","Prince William Sound, Alaska, United States",1964 Alaska earthquake,9.2
2,3,"December 26, 2004","Indian Ocean, Sumatra, Indonesia",2004 Indian Ocean earthquake,9.1–9.3
3,4,"March 11, 2011","Pacific Ocean, Tōhoku region, Japan",2011 Tōhoku earthquake,9.1[3]
4,5,"November 4, 1952","Kamchatka, Russian SFSR, Soviet Union",1952 Kamchatka earthquakes,9.0[4]


In [2]:
# Your code here


In [3]:
# Solution code here
data[['month','day','year']] = data['Date'].str.extract('(?P<month>[^-]+) (?P<day>[^-]+),(?P<year>[^-]+)')

data['Magnitude'] = data['Magnitude'].str.replace(r'\(est.\)','')
data['Magnitude'] = data['Magnitude'].str.replace(r'\[.*?\]','')

data[['magnitude_lower_bound','magnitude_upper_bound']] = data['Magnitude'].str.extract('(?P<magnitude_lower_bound>\d.\d).?(?P<magnitude_upper_bound>\d*.*\d*)')
data['magnitude_upper_bound'] = data['magnitude_upper_bound'].replace('',np.nan).astype(float)
data['magnitude_lower_bound'] = data['magnitude_lower_bound'].astype(float)
data['magnitude'] = data[['magnitude_lower_bound', 'magnitude_upper_bound']].mean(axis=1)

data = data[['year','month','day','magnitude']]
data.head()

,year,month,day,magnitude
0,1960,May,22,9.5
1,1964,March,27,9.2
2,2004,December,26,9.2
3,2011,March,11,9.1
4,1952,November,4,9.0


# Question 2: Data Manipulation
Amy wanted a new bed frame from Ikea. Before going to the nearby store, she browsed online to find some Ikea bed frames that she quite liked. When she's at the store, some bed frames were actually out of stock, and she was only able to find part of her selections. 

In [4]:
import pandas as pd
import numpy as np

online_df=pd.DataFrame([["NORDLI","wood","queen"],
        ["KOPARDAL","metal","queen"],
        ["NEIDEN","wood","full"],
        ["TARVA","wood","queen"],
        ["SLATTUM","metal","full"],
        ["BJÖRKSNÄS","wood","king"],
        ["ASKVOLL","wood","full"]], columns=["name","material","size"])

store_df=pd.DataFrame([["NEIDEN",99,"natural"],
          ["NORDLI",399,"white/black"],
          ["BJÖRKSNÄS",549,"natural"],
          ["TARVA",149,"natural"],
          ["KOPARDAL",279, "dark_grey"],
          ["SLATTUM",129,"grey"]], columns=["name","price","color"])

display(online_df)
display(store_df)

,name,material,size
0,NORDLI,wood,queen
1,KOPARDAL,metal,queen
2,NEIDEN,wood,full
3,TARVA,wood,queen
4,SLATTUM,metal,full
5,BJÖRKSNÄS,wood,king
6,ASKVOLL,wood,full


,name,price,color
0,NEIDEN,99,natural
1,NORDLI,399,white/black
2,BJÖRKSNÄS,549,natural
3,TARVA,149,natural
4,KOPARDAL,279,dark_grey
5,SLATTUM,129,grey


## Part A
Write a function to return the mean and standard deviations (from `numpy`) of the average price of a bed frame by size. Only include bed frames that Amy found in store.

In [5]:
# Your code here


In [6]:
# Solution code here
full=pd.merge(online_df,store_df,left_on='name',right_on='name', how="outer")
full.groupby("size").agg([np.nanmean,np.nanstd])

price            
          nanmean      nanstd
size                         
full   114.000000   21.213203
king   549.000000         NaN
queen  275.666667  125.033329

## Part B
On average, what is the most expensive and least expensive size/material combination? Build a dataframe where each column is labeled as to a material type and the values of the cells are average price of different bed frame sizes (so row index values should be bedframe size)

In [7]:
# Your code here

In [8]:
full.pivot_table(values="price", index="size", columns="material", aggfunc={np.nanmean})

nanmean       
material   metal   wood
size                   
full       129.0   99.0
king         NaN  549.0
queen      279.0  274.0